In [1]:
import numpy as np
import tensorflow as tf

from os import path
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter

from PIL import Image
import glob

In [2]:
np.random.seed(0)

In [3]:
sample_names = 'open-images'
record_dir = path.join('..', '..', 'data', 'datasets', 'records', 'open-images')
dataset_dir = path.join('..', '..', 'data', 'datasets', 'raw-images','open-images')
files = glob.glob(path.join(dataset_dir, '*'))

obj_dims = (648, 486)

# fwhm in pixels, for convolution with Gaussian
fwhm_pixels = np.array([2, 2])

In [4]:
np.random.shuffle(files)

In [5]:
def normalize(im):
    """
    Normalizes im from 0 to 1.
    """
    im_max = np.max(im)
    im_min = np.min(im)
    return (im - im_min) / (im_max - im_min)

def _create_example(plane):
    """
    Creates and returns tf.Example from a given numpy array.
    """
    plane_feature = tf.train.Feature(float_list=tf.train.FloatList(value=plane.ravel()))
    feature = {
        'plane': plane_feature
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [6]:
for i in range(len(files)):
    sample = Image.open(files[i])
    sample = sample.convert('L')
    sample = sample.resize(np.flip(obj_dims))
    
    sample = np.asarray(sample)
    
    fwhm_scaled = fwhm_pixels
    sigmas = fwhm_scaled / np.sqrt(8 * np.log(2))
    sample = gaussian_filter(sample, sigmas)
    sample = normalize(sample)
    
    sample.astype(np.float16)
    
    record_file = path.join(record_dir, sample_names + '-%.5d' % i + '.tfrecord')
    with tf.io.TFRecordWriter(record_file) as writer:
        tf_example = _create_example(sample)
        writer.write(tf_example.SerializeToString())

/home/rshuai/.local/lib/python3.6/site-packages/PIL/Image.py:2731: DecompressionBombWarning: Image size (144649784 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
